# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv = "output_data/cities.csv"

weather_df = pd.read_csv(weather_csv)

weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Saint George,37.1041,-113.5841,97.39,29,0,20.71,US,1659487398
1,1,Hofn,64.2539,-15.2082,48.69,73,49,11.41,IS,1659487664
2,2,Shimanovsk,52.0000,127.7000,76.48,59,0,5.99,RU,1659487665
3,3,Dolinsk,47.3500,142.8000,67.32,72,17,8.37,RU,1659487665
4,4,Sept-Îles,50.2001,-66.3821,68.00,91,75,5.75,CA,1659487657


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
weather_df = weather_df.dropna()
locations = weather_df[["Lat", "Lng"]].astype(float)
humidity = weather_df["Humidity"].astype(float)

In [4]:
# Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather_df = weather_df

# A max temperature lower than 80 degrees but higher than 70
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Max Temp"] < 80) & (ideal_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Wind Speed"] < 10]

# Zero Cloudiness
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Cloudiness"] == 0]

ideal_weather_df = ideal_weather_df.dropna()

ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,Shimanovsk,52.0000,127.7000,76.48,59,0,5.99,RU,1659487665
16,16,Albany,42.6001,-73.9662,76.59,62,0,8.48,US,1659487506
23,23,Mahon,39.8885,4.2658,75.76,78,0,2.30,ES,1659487670
45,45,Jamestown,42.0970,-79.2353,71.19,68,0,6.91,US,1659487404
125,125,Rome,43.2128,-75.4557,70.52,86,0,4.74,US,1659487697


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into variable named hotel_df 

hotel_df = ideal_weather_df

# Setting parameters
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index,row in hotel_df.iterrows():
    # Getting latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Changing Locations to look for
    params["location"] = f"{lat},{lng}"
    
    #API Url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Creating URL and API request
    print(f"Accessing results for {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("-------------------------------")
    

Accessing results for Shimanovsk.
Closest hotel is Гостиница Домашняя.
-------------------------------
Accessing results for Albany.
Missing field/result... skipping.
-------------------------------
Accessing results for Mahon.
Closest hotel is ARTIEM Capri.
-------------------------------
Accessing results for Jamestown.
Closest hotel is Hampton Inn & Suites Jamestown.
-------------------------------
Accessing results for Rome.
Closest hotel is Wingate By Wyndham Rome.
-------------------------------
Accessing results for Minudasht.
Closest hotel is هتل قصر.
-------------------------------
Accessing results for Heihe.
Closest hotel is Hotel Strannik.
-------------------------------
Accessing results for Sault Ste. Marie.
Closest hotel is Comfort Inn.
-------------------------------
Accessing results for Porto-Vecchio.
Closest hotel is Hôtel Le Goéland - Casa Santini.
-------------------------------
Accessing results for Shiraz.
Closest hotel is Shams Hotel.
---------------------------

In [7]:
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,2,Shimanovsk,52.0000,127.7000,76.48,59,0,5.99,RU,1659487665,Гостиница Домашняя
16,16,Albany,42.6001,-73.9662,76.59,62,0,8.48,US,1659487506,NaN
23,23,Mahon,39.8885,4.2658,75.76,78,0,2.30,ES,1659487670,ARTIEM Capri
45,45,Jamestown,42.0970,-79.2353,71.19,68,0,6.91,US,1659487404,Hampton Inn & Suites Jamestown
125,125,Rome,43.2128,-75.4557,70.52,86,0,4.74,US,1659487697,Wingate By Wyndham Rome
182,182,Minudasht,37.2289,55.3747,74.77,73,0,0.00,IR,1659487712,هتل قصر
220,220,Heihe,50.2441,127.4902,75.78,50,0,1.14,CN,1659487722,Hotel Strannik
241,241,Sault Ste. Marie,46.5168,-84.3333,71.89,57,0,6.91,CA,1659487595,Comfort Inn
267,267,Porto-Vecchio,41.5910,9.2795,76.53,88,0,2.30,FR,1659487733,Hôtel Le Goéland - Casa Santini
296,296,Shiraz,29.6036,52.5388,72.84,53,0,0.00,IR,1659487740,Shams Hotel


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map

# Assign the marker layer to a variable
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

# Display figure
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))